In [1]:
import pandas as pd
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from hyperopt import hp
import numpy as np
from hyperopt import Trials, tpe, fmin
import warnings
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from nbimporter import NotebookLoader

warnings.filterwarnings("ignore")

In [2]:
ma = pd.read_csv("../16 Mycoplosma/ma.csv")
map = pd.read_csv("../16 Mycoplosma/map.csv")
marth = pd.read_csv("../16 Mycoplosma/marth.csv")
mcap = pd.read_csv("../16 Mycoplosma/mcap.csv")
mcon = pd.read_csv("../16 Mycoplosma/mcon.csv")
mcro = pd.read_csv("../16 Mycoplosma/mcro.csv")
mgal = pd.read_csv("../16 Mycoplosma/mgal.csv")
mhom = pd.read_csv("../16 Mycoplosma/mhom.csv")
mhyoJ = pd.read_csv("../16 Mycoplosma/mhyoJ.csv")
mhyo232 = pd.read_csv("../16 Mycoplosma/mhyo232.csv")
mhyo7448 = pd.read_csv("../16 Mycoplosma/mhyo7448.csv")
mm163K = pd.read_csv("../16 Mycoplosma/mm163K.csv")
mms = pd.read_csv("../16 Mycoplosma/mms.csv")
mpHF = pd.read_csv("../16 Mycoplosma/mpHF.csv")
mpM = pd.read_csv("../16 Mycoplosma/mpM.csv")
ms53 = pd.read_csv("../16 Mycoplosma/ms53.csv")

In [3]:
def generate_train_test_dataset(index):
    dataset_names = [ma, map, marth, mcap, mcon, mcro, mgal, mhom, mhyoJ, mhyo232, mhyo7448, mm163K, mms, mpHF, mpM, ms53]
    merged_train = dataset_names[:index]+dataset_names[index+1:]
    train_data = pd.concat(merged_train)
    test_data = dataset_names[index]
    return train_data,test_data

In [4]:
def handle_dataset(dataset):
    columns_to_drop = ['identity','name', 'Amino acid', 'protein_sequence', 'essentiality']
    Xs = dataset.drop(columns = columns_to_drop)
    y = dataset['essentiality']
    return Xs,y

In [5]:
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)

In [14]:
from sklearn.linear_model import LassoCV

lassoCV = LassoCV(cv=20,random_state=10)
all_feature_names = ["GC_Content","CAI","A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","nSE2","nSE3","nGE2","nGE3"]

In [7]:
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 700, 1),
    'max_depth': hp.quniform('max_depth', 3, 20, 1),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5),
}

In [10]:

from sklearn.metrics import accuracy_score

def rf_ac_cv(params):
    params = {
    'n_estimators': int(params['n_estimators']), 
    'max_depth': int(params['max_depth']), 
    'min_samples_split': params['min_samples_split'],
    'min_samples_leaf': params['min_samples_leaf']
}
    scorer = make_scorer(accuracy_score)
    model = RandomForestClassifier(random_state=42, **params)
    score = -cross_val_score(model, X_resampled, y_resampled, cv=5,scoring=scorer, ).mean()
    return score

In [11]:
from joblib import load
from sklearn.feature_selection import RFE

scores = []
score = 0
models = []
model_rf_fs = load("../../model/RF_model.joblib")
for i in range(0,15):
    train_data, test_data = generate_train_test_dataset(i)
    Xs_train,y_train = handle_dataset(train_data)
    Xs_test,y_test = handle_dataset(test_data)
    rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
    rfe = RFE(model_rf_fs, n_features_to_select=20)
    rfe.fit(X_resampled, y_resampled)
    X_resampled = rfe.transform(X_resampled)  # For training data
    Xs_test = rfe.transform(Xs_test)  # For testing data
    lassoCV.fit(X_resampled,y_resampled)
    selected_features = [feature for feature, coef in zip(all_feature_names, lassoCV.coef_) if coef !=0]
    trials = Trials()
    selected_feature_indices = [all_feature_names.index(feature) for feature in selected_features]
    X_resampled = X_resampled[:, selected_feature_indices]
    Xs_test = Xs_test[:,selected_feature_indices]
    best=fmin(fn=rf_ac_cv, # function to optimize
              space=space, 
              algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
              max_evals=10, # maximum number of iterations
              trials=trials, # logging
              rstate=np.random.default_rng(42) # fixing random state for the reproducibility
    )
    model = RandomForestClassifier(random_state=42, n_estimators=int(best['n_estimators']),max_depth=int(best['max_depth']),min_samples_split=best['min_samples_split'],min_samples_leaf=best['min_samples_leaf'])
    models.append(model)
    model.fit(X_resampled,y_resampled)
    tpe_test_score=accuracy_score(y_test, model.predict(Xs_test))
    scores.append(tpe_test_score)
    print(tpe_test_score)
    if score < tpe_test_score:
        score = tpe_test_score
        best_model = model

100%|██████████| 10/10 [00:20<00:00,  2.02s/trial, best loss: -0.7944888366627497]
0.8992042440318302
100%|██████████| 10/10 [00:19<00:00,  1.98s/trial, best loss: -0.7956043956043957]
0.8530020703933747
100%|██████████| 10/10 [00:21<00:00,  2.17s/trial, best loss: -0.8050859028330869]
0.6383647798742138
100%|██████████| 10/10 [00:22<00:00,  2.22s/trial, best loss: -0.7827092569841354]
0.9795918367346939
100%|██████████| 10/10 [00:21<00:00,  2.18s/trial, best loss: -0.7930308780403658]
0.8803680981595092
100%|██████████| 10/10 [00:20<00:00,  2.09s/trial, best loss: -0.7899204086481351]
0.9322033898305084
100%|██████████| 10/10 [00:21<00:00,  2.20s/trial, best loss: -0.7814571092831963]
0.6561743341404358
100%|██████████| 10/10 [00:27<00:00,  2.75s/trial, best loss: -0.7941348973607039]
0.9483870967741935
100%|██████████| 10/10 [00:28<00:00,  2.84s/trial, best loss: -0.8380589373079987]
0.7861271676300579
100%|██████████| 10/10 [00:27<00:00,  2.77s/trial, best loss: -0.7699653937222998]

In [12]:
print(score)

0.9900249376558603


In [17]:
train_data, test_data = generate_train_test_dataset(13)
Xs_train,y_train = handle_dataset(train_data)
X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
rfe = RFE(model_rf_fs, n_features_to_select=20)
rfe.fit(X_resampled, y_resampled)
X_resampled = rfe.transform(X_resampled)
lassoCV.fit(X_resampled,y_resampled)
selected_feature_indices = rfe.get_support(indices=True)
print(selected_feature_indices)
selected_feature_names = [all_feature_names[index-1] for index in selected_feature_indices]
selected_features = [feature for feature, coef in zip(selected_feature_names, lassoCV.coef_) if coef !=0]
print("Selected feature RFE:", selected_feature_names)
print("Selected feature Lasso:", selected_features)

[ 1  2  3  4  5  6  7 10 12 13 14 16 18 19 21 22 23 24 25 26]
Selected feature RFE: ['GC_Content', 'CAI', 'A', 'R', 'N', 'D', 'C', 'G', 'I', 'L', 'K', 'F', 'S', 'T', 'Y', 'V', 'nSE2', 'nSE3', 'nGE2', 'nGE3']
Selected feature Lasso: ['GC_Content', 'CAI', 'A', 'R', 'N', 'D', 'G', 'K', 'F', 'S', 'T', 'Y', 'V', 'nSE2', 'nSE3', 'nGE2', 'nGE3']


In [18]:
from joblib import dump

dump(best_model, '../../model/RF_model_FSS.joblib')

['../../model/RF_model_FSS.joblib']

In [12]:
train_data, test_data = generate_train_test_dataset(12)
Xs_train,y_train = handle_dataset(train_data)
Xs_test,y_test = handle_dataset(test_data)
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
lassoCV.fit(X_resampled,y_resampled)
selected_features = [feature for feature, coef in zip(all_feature_names, lassoCV.coef_) if coef !=0]
print(selected_features)

['GC_Content', 'CAI', 'A', 'N', 'D', 'H', 'M', 'nSE3', 'nGE3']
